In [1]:
import problema_espacio_estados as probee
import búsqueda_espacio_estados as búsqee
import copy
from tkinter import *

In [2]:
class Hitori:
    def __init__(self, casillas):
        self.casillas = casillas
    
    def tamaño_hor(self):
        return len(self.casillas[0])
    
    def tamaño_ver(self):
        return len(self.casillas)
    
    def valor_casilla(self, f, c):
        return self.casillas[f][c]    
    
    def __str__(self):
        cadena = str(self.casillas)
        
        return cadena

In [114]:
class Poner_negro(probee.Acción):
    def __init__(self, i, j):
        nombre = 'posición {},{}'.format(i, j)
        super().__init__(nombre)
        self.f = i
        self.c = j
    
    def es_negro(self, estado, f, c):
        return bool(Hitori.valor_casilla(estado,f,c) == 0)
    
    def es_blanco(self, f, c):
        return not bool(Hitori.valor_casilla(self,f,c) == 0)
    
    def negro_colindante(self, estado):
        arriba = False
        abajo = False
        izquierda = False
        derecha = False
        
        if self.f < (Hitori.tamaño_ver(estado)-1):
            abajo = self.es_negro(estado, (self.f)+1, self.c)
         
        if self.f > 0:
            arriba = self.es_negro(estado, (self.f)-1, self.c)
            
        if self.c < (Hitori.tamaño_hor(estado)-1):
            derecha = self.es_negro(estado, self.f, (self.c)+1)
            
        if self.c > 0:
            izquierda = self.es_negro(estado, self.f, (self.c)-1)
            
        return arriba or abajo or izquierda or derecha
    
    def duplicado_fila(self, estado):
        tamaño = Hitori.tamaño_hor(estado)
        limite_izq = self.c
        limite_der = tamaño
        for i in range(0, limite_izq):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                if Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
                
        for j in range((self.c)+1, limite_der):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                 if Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
                
    
    def duplicado_columna(self, estado):
        tamaño = Hitori.tamaño_ver(estado)
        limite_arr = self.f
        limite_aba = tamaño
        for i in range(0, limite_arr):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                if Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
                
        for j in range((self.f)+1, limite_aba):
            if Hitori.valor_casilla(estado, self.f,self.c) != 0:
                 if Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, self.f, self.c):
                    return True
            
    
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |     |  0     |
    #          -----------------------
    #          |       |  x  |        |
    #          -----------------------
    #
    #
    #
    #
    def abajo_encierra(self, estado):
        arriba = True
        arriba_derecha = True
        arriba_izquierda = True
        
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba_izquierda = False        
                
        if self.f > 0 and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba_derecha = False
                
        if self.f > 1: 
            if not self.es_negro(estado, (self.f)-2, self.c):
                arriba = False
                
        if self.f == 0:
            arriba = False
            arriba_derecha = False
            arriba_izquierda = False
            
            
        return arriba and arriba_derecha and arriba_izquierda    
        
    #
    #
    #           ------------------------
    #          |       |  x  |        |
    #          ------------------------
    #          |    0  |     |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def arriba_encierra(self, estado):
        abajo = True
        abajo_derecha = True
        abajo_izquierda = True
        
        if self.f < Hitori.tamaño_ver(estado)-1 and self.c > 0:
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo_izquierda = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo_derecha = False
                
        if self.f < (Hitori.tamaño_ver(estado)-2): 
            if not self.es_negro(estado, (self.f)+2, self.c):
                abajo = False
                
        if self.f == (Hitori.tamaño_ver(estado)-1):
            abajo = False
            abajo_derecha = False
            abajo_izquierda = False
            
            
        return abajo and abajo_derecha and abajo_izquierda    
      
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    0  |     |  x     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def derecha_encierra(self, estado):
        izquierda = True
        arriba = True
        abajo = True
        
        if self.f > 0 and self.c > 0:
            if not self.es_negro(estado, (self.f)-1, (self.c)-1):
                arriba = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c > 0:      
            if not self.es_negro(estado, (self.f)+1, (self.c)-1):
                abajo = False
                
        if self.c > 1: 
            if not self.es_negro(estado, (self.f), self.c-2):
                izquierda = False
                
        if self.c == 0:
            izquierda = False
            arriba = False
            abajo = False
            
            
        return izquierda and arriba and abajo  
    
    #
    #
    #           ------------------------
    #          |       |  0  |        |
    #          ------------------------
    #          |    x  |     |  0     |
    #          -----------------------
    #          |       |  0  |        |
    #          -----------------------
    #
    #
    #
    #
    def izquierda_encierra(self, estado):
        derecha = True
        arriba = True
        abajo = True
        
        if self.f > 0 and self.c < Hitori.tamaño_hor(estado)-1:
            if not self.es_negro(estado, (self.f)-1, (self.c)+1):
                arriba = False        
                
        if self.f < (Hitori.tamaño_ver(estado)-1) and self.c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, (self.f)+1, (self.c)+1):
                abajo = False
                
        if self.c < Hitori.tamaño_hor(estado)-2: 
            if not self.es_negro(estado, (self.f), self.c+2):
                derecha = False
                
        if self.c == (Hitori.tamaño_hor(estado)-1):
            derecha = False
            arriba = False
            abajo = False
            
            
        return derecha and arriba and abajo    
     
        
    #Si se cumple alguna de las restricciones, poner negro la casilla provoca de se encierre una casilla
    def encierra(self, estado):
        return bool(self.abajo_encierra(estado) 
                    or self.arriba_encierra(estado) 
                    or self.izquierda_encierra(estado) 
                    or self.derecha_encierra(estado))
    
    def diagonal_arriba_derecha(self, estado):
        limite = min(self.f, Hitori.tamaño_hor(estado)-1-self.c)
        result = False
        if self.f != 0 and (self.c != Hitori.tamaño_hor(estado)-1):
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f-ac, self.c+ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result
    
    def diagonal_arriba_izquierda(self, estado):
        limite = min(self.f, self.c)
        result = False
        if self.f != 0 and self.c != 0:
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f-ac, self.c-ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result   
    
    def diagonal_abajo_derecha(self, estado):
        limite = min(Hitori.tamaño_ver(estado)-1-self.f, Hitori.tamaño_hor(estado)-1-self.c)
        result = False
        if (self.f != Hitori.tamaño_ver(estado)-1) and (self.c != Hitori.tamaño_hor(estado)-1):
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f+ac, self.c+ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result
      
    def diagonal_abajo_izquierda(self, estado):
        limite = min(Hitori.tamaño_ver(estado)-1-self.f, self.c)
        result = False
        if (self.f != Hitori.tamaño_ver(estado)-1) and self.c != 0:
            for ac in range(1,limite+1):
                if not self.es_negro(estado, self.f+ac, self.c-ac):
                    result = False
                    break
                else: 
                    result = True
                    
        else:
            result = False
            
        return result 
    
    def parte_matriz(self, estado):
        arriba_derecha = self.diagonal_abajo_derecha(estado)
        arriba_izquierda = self.diagonal_abajo_izquierda(estado)
        abajo_derecha = self.diagonal_abajo_derecha(estado)
        abajo_izquierda = self.diagonal_abajo_izquierda(estado)
        
        return ((arriba_derecha and arriba_izquierda) 
                or (arriba_derecha and abajo_derecha) 
                or (arriba_derecha and abajo_izquierda) 
                or (arriba_izquierda and arriba_derecha) 
                or (arriba_izquierda and abajo_derecha) 
                or (abajo_derecha and abajo_izquierda))
        
    def es_aplicable(self, estado):
        return bool(not self.negro_colindante(estado)
                    and (self.duplicado_fila(estado) or self.duplicado_columna(estado))
                    and not self.encierra(estado))
                    #and not self.parte_matriz(estado))
        
    def duplicado_fila_aux(self, estado, f, c):
        num = Hitori.valor_casilla(estado, f, c)
        if estado.casillas[f].count(num) > 1:
            return True
                
    
    def duplicado_columna_aux(self, estado, f, c):
        tamaño = Hitori.tamaño_ver(estado)
        limite_izq = f
        limite_der = tamaño
        for i in range(0, limite_izq):
            if Hitori.valor_casilla(estado, f, c) != 0:
                if Hitori.valor_casilla(estado, i, c) == Hitori.valor_casilla(estado, f, c):
                    return True
                
        for j in range(f+1, limite_der):
            if Hitori.valor_casilla(estado, f, c) != 0:
                 if Hitori.valor_casilla(estado, j, c) == Hitori.valor_casilla(estado, f, c):
                    return True
                
    def negro_colindante_aux(self, estado, f, c):
        arriba = False
        abajo = False
        izquierda = False
        derecha = False
        
        if f < (Hitori.tamaño_ver(estado)-1):
            abajo = self.es_negro(estado, f+1, c)
         
        if f > 0:
            arriba = self.es_negro(estado, f-1, c)
            
        if c < (Hitori.tamaño_hor(estado)-1):
            derecha = self.es_negro(estado, f, c+1)
            
        if c > 0:
            izquierda = self.es_negro(estado, f, c-1)
            
        return arriba or abajo or izquierda or derecha
    
    def abajo_encierra_aux(self, estado, f, c):
        arriba = True
        arriba_derecha = True
        arriba_izquierda = True
        
        if f > 0 and c > 0:
            if not self.es_negro(estado, f-1, c-1):
                arriba_izquierda = False        
                
        if f > 0 and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f-1, c+1):
                arriba_derecha = False
                
        if f > 1: 
            if not self.es_negro(estado, f-2, c):
                arriba = False
                
        if f == 0:
            arriba = False
            arriba_derecha = False
            arriba_izquierda = False
            
            
        return arriba and arriba_derecha and arriba_izquierda
    
    def arriba_encierra_aux(self, estado, f, c):
        abajo = True
        abajo_derecha = True
        abajo_izquierda = True
        
        if f < Hitori.tamaño_ver(estado)-1 and c > 0:
            if not self.es_negro(estado, f+1, c-1):
                abajo_izquierda = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f+1, c+1):
                abajo_derecha = False
                
        if f < (Hitori.tamaño_ver(estado)-2): 
            if not self.es_negro(estado, f+2, c):
                abajo = False
                
        if f == (Hitori.tamaño_ver(estado)-1):
            abajo = False
            abajo_derecha = False
            abajo_izquierda = False
            
            
        return abajo and abajo_derecha and abajo_izquierda    
      
    
    def derecha_encierra_aux(self, estado, f, c):
        izquierda = True
        arriba = True
        abajo = True
        
        if f > 0 and c > 0:
            if not self.es_negro(estado, f-1, c-1):
                arriba = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c > 0:      
            if not self.es_negro(estado, f+1, c-1):
                abajo = False
                
        if c > 1: 
            if not self.es_negro(estado, f, c-2):
                izquierda = False
                
        if c == 0:
            izquierda = False
            arriba = False
            abajo = False
            
            
        return izquierda and arriba and abajo  
    
  
    def izquierda_encierra_aux(self, estado, f, c):
        derecha = True
        arriba = True
        abajo = True
        
        if f > 0 and c < Hitori.tamaño_hor(estado)-1:
            if not self.es_negro(estado, f-1, c+1):
                arriba = False        
                
        if f < (Hitori.tamaño_ver(estado)-1) and c < (Hitori.tamaño_hor(estado)-1):      
            if not self.es_negro(estado, f+1, c+1):
                abajo = False
                
        if c < Hitori.tamaño_hor(estado)-2: 
            if not self.es_negro(estado, f, c+2):
                derecha = False
                
        if c == (Hitori.tamaño_hor(estado)-1):
            derecha = False
            arriba = False
            abajo = False
            
            
        return derecha and arriba and abajo 
    
    def encierra_aux(self, estado, f, c):
        return bool(self.abajo_encierra_aux(estado, f, c) 
                    or self.arriba_encierra_aux(estado, f, c) 
                    or self.izquierda_encierra_aux(estado, f, c) 
                    or self.derecha_encierra_aux(estado, f, c))
    
    def aplicar(self, estado):
        nuevo_estado = copy.deepcopy(estado)
        nuevo_estado.casillas[self.f][self.c] = 0
        
        #Comprobamos que existe una posición a la derecha
        if self.c < (Hitori.tamaño_hor(estado)-1):
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, self.f, (self.c)+1):
                tamaño = Hitori.tamaño_hor(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, i)
                       and not self.encierra_aux(nuevo_estado, self.f, i)):
                        nuevo_estado.casillas[self.f][i] = 0

                for j in range((self.c)+2, limite_der):
                    if (Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, j)
                       and not self.encierra_aux(nuevo_estado, self.f, j)):
                        nuevo_estado.casillas[self.f][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, self.f, (self.c)+1):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, (self.c)+1) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c+1)
                       and not self.encierra_aux(nuevo_estado, i, self.c+1)):
                        nuevo_estado.casillas[i][self.c+1] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, (self.c)+1) == Hitori.valor_casilla(estado, self.f, self.c+1)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c+1)
                       and not self.encierra_aux(nuevo_estado, j, self.c+1)):
                        nuevo_estado.casillas[j][self.c+1] = 0
                        
        #Comprobamos que existe una posición a la izquierda
        if self.c > 0:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, self.f, (self.c)-1):
                tamaño = Hitori.tamaño_hor(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq-1):
                    if (Hitori.valor_casilla(estado, self.f, i) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, i)
                       and not self.encierra_aux(nuevo_estado, self.f, i)):
                        nuevo_estado.casillas[self.f][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, self.f, j) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, self.f, j)
                       and not self.encierra_aux(nuevo_estado, self.f, j)):
                        nuevo_estado.casillas[self.f][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, self.f, (self.c)-1):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, (self.c)-1) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c-1)
                       and not self.encierra_aux(nuevo_estado, i, self.c-1)):
                        nuevo_estado.casillas[i][self.c-1] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, (self.c)-1) == Hitori.valor_casilla(estado, self.f, self.c-1)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c-1)
                       and not self.encierra_aux(nuevo_estado, j, self.c-1)):
                        nuevo_estado.casillas[j][self.c-1] = 0
                        
        #Comprobamos que existe una posición de arriba
        if self.f > 0:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, (self.f)-1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, (self.f)-1, i) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)-1, i)
                       and not self.encierra_aux(nuevo_estado, (self.f)-1, i)):
                        nuevo_estado.casillas[self.f-1][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, (self.f)-1, j) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)-1, j)
                       and not self.encierra_aux(nuevo_estado, (self.f)-1, j)):
                        nuevo_estado.casillas[self.f-1][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, (self.f)-1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr-1):
                    if (Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c)
                       and not self.encierra_aux(nuevo_estado, i, self.c)):
                        nuevo_estado.casillas[i][self.c] = 0

                for j in range((self.f)+1, limite_aba):
                    if (Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, (self.f)-1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c)
                       and not self.encierra_aux(nuevo_estado, j, self.c)):
                        nuevo_estado.casillas[j][self.c] = 0
                        
        #Comprobamos que existe una posición de abajo
        if self.f < Hitori.tamaño_ver(estado)-1:
            #Comprobamos que hay duplicados en la fila
            if self.duplicado_fila_aux(estado, (self.f)+1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                #No llegaría al que ponemos en negro
                limite_izq = self.c
                limite_der = tamaño
                for i in range(0, limite_izq):
                    if (Hitori.valor_casilla(estado, (self.f)+1, i) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)+1, i)
                       and not self.encierra_aux(nuevo_estado, (self.f)+1, i)):
                        nuevo_estado.casillas[self.f+1][i] = 0

                for j in range((self.c)+1, limite_der):
                    if (Hitori.valor_casilla(estado, (self.f)+1, j) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, (self.f)+1, j)
                       and not self.encierra_aux(nuevo_estado, (self.f)+1, j)):
                        nuevo_estado.casillas[self.f+1][j] = 0
            #Comprobamos que hay duplicados en columna
            if self.duplicado_columna_aux(estado, (self.f)+1, self.c):
                tamaño = Hitori.tamaño_ver(estado)
                limite_arr = self.f
                limite_aba = tamaño
                for i in range(0, limite_arr):
                    if (Hitori.valor_casilla(estado, i, self.c) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, i, self.c)
                       and not self.encierra_aux(nuevo_estado, i, self.c)):
                        nuevo_estado.casillas[i][self.c] = 0

                for j in range((self.f)+2, limite_aba):
                    if (Hitori.valor_casilla(estado, j, self.c) == Hitori.valor_casilla(estado, (self.f)+1, self.c)
                       and not self.negro_colindante_aux(nuevo_estado, j, self.c)
                       and not self.encierra_aux(nuevo_estado, j, self.c)):
                        nuevo_estado.casillas[j][self.c] = 0    
                        
            
        return nuevo_estado

In [352]:
class ProblemaHitori(probee.ProblemaEspacioEstados):
    def __init__(self, estado_inicial):
        acciones = [Poner_negro (i,j) for i in range(0, Hitori.tamaño_hor(estado_inicial)) for j in range(0, Hitori.tamaño_ver(estado_inicial))]
        estado_inicial = estado_inicial
        super().__init__(acciones, estado_inicial)
        
    def duplicado_fila(self, estado):
        
        result = False
        
        for acVer in range(0, Hitori.tamaño_ver(estado)):
        
            if result != True:
            
                for acHor in range(0, Hitori.tamaño_hor(estado)):

                    if result != True:

                        if Hitori.valor_casilla(estado, acVer, acHor) != 0:

                            for ac in range(acHor+1, Hitori.tamaño_hor(estado)):

                                if Hitori.valor_casilla(estado, acVer, acHor) == Hitori.valor_casilla(estado, acVer, ac):
                                    result = True
                                    break

                                else:
                                    result = False
                    else:  
                        break
                        
            else:
                break
                
        return result  
    
    def duplicado_fila2(self, estado):
        
        result = False
        
        num_max = max(Hitori.tamaño_hor(estado),Hitori.tamaño_ver(estado))
        for fila in range(0, Hitori.tamaño_ver(estado)):
            
            if not result:
                for num in range(1, num_max+1):
                    if estado.casillas[fila].count(num) > 1:
                        result = True
                        break
            else:
                break
        
        return result
        
        
    def duplicado_columna(self, estado):
        
        result = False
        
        for acHor in range(0, Hitori.tamaño_hor(estado)):
        
            if result != True:
            
                for acVer in range(0, Hitori.tamaño_ver(estado)):

                    if result != True:

                        if Hitori.valor_casilla(estado, acVer, acHor) != 0:

                            for ac in range(acVer+1, Hitori.tamaño_ver(estado)):

                                if Hitori.valor_casilla(estado, acVer, acHor) == Hitori.valor_casilla(estado, ac, acHor):
                                    result = True
                                    break

                                else:
                                    result = False
                    else:  
                        break
                        
            else:
                break
                
        return result    
                    
                
        
    
    def es_estado_final(self, estado):
        return bool(not self.duplicado_fila2(estado) and not self.duplicado_columna(estado))

In [621]:
estado_inicial = Hitori([[3,2,9,1,8,6,5,4,6],[3,8,1,2,7,5,9,5,4],[9,4,5,4,6,3,3,8,1],[9,4,1,9,5,7,5,1,2],[1,9,7,5,2,8,4,7,5],[6,4,2,8,4,9,5,7,1],[1,1,6,1,9,4,8,2,3],[2,6,4,1,3,1,7,8,8],[9,7,8,3,1,8,8,6,9]])

In [622]:
p1 = ProblemaHitori(estado_inicial)

In [623]:
b_nueva = búsqee.BúsquedaEnProfundidad()

In [ ]:
%%timeit -n1 -r1

b_nueva.buscar(p1)SSS